<a href="https://colab.research.google.com/github/sepehrkr/Deep-Learning-Course/blob/main/Question2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install hazm

In [3]:
import hazm
import numpy as np
import json
import os
import glob

import numpy as np
import pandas as pd

from tqdm import tqdm

from pathlib import Path

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from transformers import AutoTokenizer, AutoConfig, AutoModelWithLMHead
from transformers import AutoTokenizer, GPT2LMHeadModel, GPT2Config

from IPython import display
from transformers import GPT2LMHeadModel, GPT2Config
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup


## Preprocessing

In [4]:
normalizer = hazm.Normalizer(persian_numbers=False)
normalize = lambda text: normalizer.normalize(text)

with open('./ferdousi.txt','r') as f:
  data = f.read()

data = data.splitlines()[2:]
if len(data) % 2 != 0:
  data = data[:-1]
data = np.array(data)
df = pd.DataFrame()
df['first beyt'] = data[range(0,len(data),2)]
df['second beyt'] = data[range(1,len(data),2)]
df['first beyt'] = df['first beyt'].apply(normalize)
df['second beyt'] = df['second beyt'].apply(normalize)
df['texts'] = df['first beyt'] + "<|startoftext|>" + df['second beyt']

In [ ]:
model_name_or_path = "HooshvareLab/gpt2-fa"

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    bos_token='<BOS>',
    eos_token='<EOS>',
    pad_token='<PAD>',
    unk_token='<UNK>'
)
tokenizer.add_special_tokens({
    "bos_token": '<BOS>',
    "eos_token": '<EOS>',
    "pad_token": '<PAD>',
    "unk_token": '<UNK>'
})

config = AutoConfig.from_pretrained(
    model_name_or_path,
    bos_token_id=tokenizer("<BOS>")["input_ids"][0],
    eos_token_id=tokenizer("<EOS>")["input_ids"][0],
    pad_token_id=tokenizer("<PAD>")["input_ids"][0],
    unk_token_id=tokenizer("<UNK>")["input_ids"][0],
)

tokenizer.save_pretrained("/content/gpt2/")
config.save_pretrained("/content/gpt2/")

!wget "https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/pytorch_model.bin" -P /content/gpt2/
!wget "https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/tokenizer.json" -P /content/gpt2/

tokenizer = AutoTokenizer.from_pretrained(
    './gpt2/',
    bos_token='<BOS>',
    eos_token='<EOS>',
    pad_token='<PAD>',
    unk_token='<UNK>'
)

## Creating Dataset

In [6]:
class PoetDataset(Dataset):

    def __init__(self, texts, tokenizer, max_length=1024):

        self.tokenizer = tokenizer  # the gpt2 tokenizer we instantiated
        self.encoding = []
        self.mask = []

        for text in texts:
          encoding_dict = tokenizer('<BOS>' + text + '<EOS>',truncation=True, max_length=max_length, padding="max_length")
          self.encoding.append(torch.tensor(encoding_dict['input_ids']))
          self.mask.append(torch.tensor(encoding_dict['attention_mask']))

    def __len__(self):
        return len(self.encoding)

    def __getitem__(self, idx):
        return self.encoding[idx], self.mask[idx]

In [25]:
max_length = 30
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = PoetDataset(df['texts'], tokenizer, max_length)
trainset, testset = random_split(dataset, [0.8, 0.2])

trainLoader = DataLoader(trainset, batch_size, shuffle=True)
testLoader = DataLoader(testset, batch_size, shuffle=True)

## Training Model

In [12]:
epochs_num = 3
warmup_num = 100
total_steps = len(trainLoader) * epochs_num

configuration = GPT2Config.from_pretrained('./gpt2', output_hidden_states=False)
model = GPT2LMHeadModel.from_pretrained("./gpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

optimizer = AdamW(model.parameters(),lr=5e-4, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, warmup_num, total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
training_stats = []

for epoch in range(epochs_num):

    print(f'epoch {epoch}: ')

    total_train_loss = 0

    model.train()

    for i, (encoding, mask) in tqdm(enumerate(trainLoader), total=len(trainLoader)):

        encoding = encoding.to(device)
        mask = mask.to(device)

        model.zero_grad()

        outputs = model(encoding, labels=encoding, attention_mask=mask, token_type_ids=None)

        loss = outputs[0]

        total_train_loss += loss.item()

        # Get sample every 100 batches.
        if i % 200 == 0 and not i == 0:

            model.eval()

            first_beyt = df.loc[np.random.randint(0, len(df))]['first beyt']
            sample_input = first_beyt + "<|startoftext|>"
            sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
            sample_input_ids = sample_input_ids.to(device)

            sample_outputs = model.generate(
                input_ids=sample_input_ids,
                do_sample=True,
                top_k=50,
                max_length=50,
                top_p=0.95,
                num_return_sequences=1
            )
            for i, sample_output in enumerate(sample_outputs):
                gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
                gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
                gen_sample_output = gen_sample_output.replace("<BOS>", "")
                gen_sample_output = gen_sample_output.replace("<EOS>", "")
                gen_sample_output = gen_sample_output.replace("<UNK>", "")

                print(f'Example output: {gen_sample_output}')

            model.train()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(trainLoader)

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for (encoding, mask) in tqdm(testLoader, total=len(testLoader)):

        encoding = encoding.to(device)
        mask = mask.to(device)

        with torch.no_grad():

            outputs = model(encoding, attention_mask=mask, labels=encoding)

            loss = outputs[0]

        total_eval_loss += loss.item()

    avg_val_loss = total_eval_loss / len(testLoader)

    print(f'Validation loss: {avg_val_loss}')


    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
        }
    )

print('training finished.')

epoch 0: 


 16%|█▌        | 201/1241 [02:46<24:48,  1.43s/it]

Example output: سر کودک مرده بینی چو شیر
کجا آمد سر بدگمان


 32%|███▏      | 401/1241 [05:26<13:42,  1.02it/s]

Example output: به آزادگان گفت یزدان سپاس
که از ما من این سخن‌های جز از من نگفت


...

Example output: بفرمود تا پرده برداشتند
ز درگاهشان شاد بگذاشتند


 97%|█████████▋| 1201/1241 [15:51<00:26,  1.52it/s]

Example output: چو آگه شد از کار خاقان چین
که سالارشان داد بر پشت زین


100%|██████████| 1241/1241 [05:18<00:00,  3.90it/s]

Validation loss: 1.0684753616042526
training finished.


## Evaluation

In [26]:
model.eval()

total_test_loss = 0
perplexity = 0
nb_eval_steps = 0

for (encoding, mask) in tqdm(testLoader, total=len(testLoader)):

    encoding = encoding.to(device)
    mask = mask.to(device)
    with torch.no_grad():
        outputs = model(encoding, attention_mask=mask, labels=encoding)
        loss = outputs[0]

    total_eval_loss += loss.item()

avg_val_loss = total_eval_loss / len(testLoader)
perplexity = 2**(-avg_val_loss)

print(f'Validation loss: {avg_val_loss}')
print(f'Perplexity: {perplexity}')

100%|██████████| 311/311 [01:50<00:00,  2.82it/s]

Validation loss: 5.481922217504005


In [27]:
perplexity = 2**(-avg_val_loss)
print(f'Perplexity: {perplexity}')

Perplexity: 0.022375717929265255


The **Perplexity** is 0.0223, thats not good becasue to the limited resoucrces we trained for 3 epochs.

In [33]:
for j in range(20):
    first_beyt = df.loc[np.random.randint(0, len(df))]['first beyt']
    sample_input = first_beyt + "<|startoftext|>"
    sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
    sample_input_ids = sample_input_ids.to(device)

    sample_outputs = model.generate(
        input_ids=sample_input_ids,
        do_sample=True,
        top_k=50,
        max_length=50,
        top_p=0.95,
        num_return_sequences=1
    )
    for i, sample_output in enumerate(sample_outputs):
        gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
        gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
        gen_sample_output = gen_sample_output.replace("<BOS>", "")
        gen_sample_output = gen_sample_output.replace("<EOS>", "")
        gen_sample_output = gen_sample_output.replace("<UNK>", "")

        print(f'Example output {j+1}:\n{gen_sample_output}')

Example output 1:
کند حلقه در گردن کنگره
چو آن مهره بر گردن کنگره
Example output 2:
نماند آنگهی جایگاه سخن
به پالیز کین بخت گم کرد بن
Example output 3:
در و دشت گفتی که زرین شدست
بران سان که گفتی ستاره شدست
Example output 4:
کمند اندرافگند و برگاشت روی
جهان شد چو چشم خروس آبگون
Example output 5:
جدا گشت زو کودکی چون پری
بدید آن بزرگی و فر و اورند شاه
Example output 6:
سپهبد فریبرز را گفت مرد
که فردا ز گردان ما باد سرد
Example output 7:
تنش پرنگار از کران تا کران
همان با درفش سران افسران
Example output 8:
یکی رزم کرد آن نه بر آرزوی
که شاهد برو جامه خسروی
Example output 9:
خروشان همی رفت نیزه بدست
همانا چو پیلی شد از پیل مست
Example output 10:
بگفت این و باد از جگر برکشید
زمین کوه گشت آهنین شد ناپدید
Example output 11:
جهاندیده گیو اندر آمد به آب
جهان کرد ازو دید چون پر آفتاب
Example output 12:
به دستان نگه کرد فرخنده سام
ابا او به سام دلیر و جوان
Example output 13:
زن و مرد ایرانیان صدهزار
همه گرد و شایسته کارزار
Example output 14:
بیامد بدرگاه سالار نو
به نزدیک بهرام یل خوار نو
Example

Not Bad!